In [6]:
import cv2
import numpy as np
from tkinter import Tk, Label, Button
from PIL import Image, ImageTk
from keras.models import model_from_json

window = Tk()
window.title("Emotion Detection")

#emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

# Load JSON and create model
# Load JSON and create model
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# Load weights into the model
emotion_model.load_weights("model/emotion_model.h5")

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}


# Start the webcam feed
cap = cv2.VideoCapture(0)  # Open the webcam (device index 0)

emotion_report = []  # Variable to store emotion predictions

while True:
    # Read frame from the webcam feed
    ret, frame = cap.read()  # Capture a frame from the webcam
    frame = cv2.resize(frame, (1280, 720))  # Resize the frame to a desired size
    if not ret:
        break
    frame = cv2.flip(frame, 1)  # Flip the frame horizontally

    face_detector_front = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')  # Load the face detector model
    gray_frame_front = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert the frame to grayscale

    # Detect faces in the frame
    num_faces = face_detector_front.detectMultiScale(gray_frame_front, scaleFactor=1.3, minNeighbors=5)  # Detect faces using the loaded model

    # Process each face detected
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)  # Draw a rectangle around the face region
        roi_gray_frame_front = gray_frame_front[y:y + h, x:x + w]  # Extract the region of interest (face) from the frame
        cropped_img_front = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame_front, (48, 48)), -1), 0)  # Preprocess the face image for emotion prediction

        # Predict the emotions
        emotion_prediction_front = emotion_model.predict(cropped_img_front)  # Predict the emotions using the loaded model
        maxindex_front = int(np.argmax(emotion_prediction_front))  # Find the index of the predicted emotion with the highest probability
        cv2.putText(frame, emotion_dict[maxindex_front], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)  # Display the predicted emotion as text on the frame

        # Append emotion to the report
        emotion_report.append(emotion_dict[maxindex_front])  # Store the predicted emotion in the emotion report list

    cv2.imshow('Emotion Detection', frame)  # Display the frame with emotion detection
    if cv2.waitKey(1) & 0xFF == ord('q'):  # Exit the loop if 'q' is pressed
        break

cap.release()  # Release the webcam
cv2.destroyAllWindows()  # Close all OpenCV windows

# Generate Emotion Report
emotion_count = {}  # Dictionary to store the count of each emotion
total_emotions = len(emotion_report)  # Total number of emotions in the report

for emotion in emotion_report:
    if emotion in emotion_count:
        emotion_count[emotion] += 1  # Increment the count if the emotion already exists in the dictionary
    else:
        emotion_count[emotion] = 1  # Initialize the count if the emotion doesn't exist in the dictionary

emotion_percentage = {}  # Dictionary to store the percentage of each emotion
for emotion, count in emotion_count.items():
    percentage = (count / total_emotions) * 100  # Calculate the percentage of each emotion
    emotion_percentage[emotion] = percentage  # Store the percentage in the dictionary

print("Emotion Report:")
for emotion, percentage in emotion_percentage.items():
    print(f"{emotion}: {percentage:.2f}%")  # Print the emotion report with percentages


Model loaded from disk
1/1 [==============================] - 0s 94ms/step
Emotion Report:
Fearful: 20.00%
Surprised: 10.00%
Neutral: 50.00%
Angry: 10.00%
Happy: 10.00%


In [3]:
import cv2
import numpy as np
from tkinter import Tk, Button, Toplevel, Label, messagebox, Frame, filedialog, Entry
from PIL import Image, ImageTk
from keras.models import model_from_json
from tkinter import ttk

# Load JSON and create model
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# Load weights into the model
emotion_model.load_weights("model/emotion_model.h5")

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

emotion_report = []  # Variable to store emotion predictions

def end_detection():
    window2.destroy()
    window.deiconify()

def start_detection():
    global cap, window2, label_img, stop_camera, emotion_report

    window.withdraw()  # Hide the first window

    cap = cv2.VideoCapture(0)
    stop_camera = False
    emotion_report = []  # Reset emotion report

    window2 = Toplevel()
    window2.title("Emotion Detection")

    background_image = Image.open("background_image.jpg")
    background_image = background_image.resize((1400, 1000), Image.LANCZOS)
    background_photo = ImageTk.PhotoImage(background_image)

    label_background = Label(window2, image=background_photo)
    label_background.place(x=0, y=0, relwidth=1, relheight=1)
    label_background.image = background_photo

    button_frame = Frame(window2, bg='white', padx=20, pady=20)
    button_frame.pack(side='bottom')

    def stop_detection():
        global stop_camera
        stop_camera = True
        window2.destroy()
        window.deiconify()

    def generate_report():
        global stop_camera, emotion_report
        stop_camera = True

        if not emotion_report:
            messagebox.showerror("Error", "No emotions detected. Please try again.")
            return

        emotion_count = {}
        total_emotions = len(emotion_report)

        for emotion in emotion_report:
            if emotion in emotion_count:
                emotion_count[emotion] += 1
            else:
                emotion_count[emotion] = 1

        emotion_percentage = {}
        for emotion, count in emotion_count.items():
            percentage = (count / total_emotions) * 100
            emotion_percentage[emotion] = percentage

        overall_mood = max(emotion_percentage, key=emotion_percentage.get)

        report_window = Toplevel()
        report_window.title("Emotion Report")

        # Set the background image for the report window
        background_image = Image.open("background_image.jpg")
        background_image = background_image.resize((1500, 1000), Image.LANCZOS)
        background_photo = ImageTk.PhotoImage(background_image)
        label_background = Label(report_window, image=background_photo)
        label_background.place(x=0, y=0, relwidth=1, relheight=1)
        label_background.image = background_photo  # Attach the image object to the label to prevent it from being garbage collected

        report_frame = Frame(report_window)
        report_frame.pack()

        for emotion, percentage in emotion_percentage.items():
            label = Label(report_frame, text=f"{emotion_dict[emotion]}: {percentage:.2f}%", font=("Arial", 16, "bold"))
            label.pack(anchor='center')

        label_overall_mood = Label(report_frame, text=f"Overall Mood: {emotion_dict[overall_mood]}", font=("Arial", 20, "bold"))
        label_overall_mood.pack(anchor='center')

        def send_report_to_therapist():
            report_window.destroy()
            details_window = Toplevel()
            details_window.title("Patient Details")

            # Set the background image for the details window
            background_image = Image.open("background_image.jpg")
            background_image = background_image.resize((1500, 1000), Image.LANCZOS)
            background_photo = ImageTk.PhotoImage(background_image)
            label_background = Label(details_window, image=background_photo)
            label_background.place(x=0, y=0, relwidth=1, relheight=1)
            label_background.image = background_photo

            details_frame = Frame(details_window)
            details_frame.pack()

            # Add labels and entry fields for patient details
            label_name = Label(details_frame, text="Name:", font=("Arial", 16, "bold"))
            label_name.pack()
            entry_name = Entry(details_frame, font=("Arial", 14))
            entry_name.pack()

            label_contact = Label(details_frame, text="Contact No:", font=("Arial", 16, "bold"))
            label_contact.pack()
            entry_contact = Entry(details_frame, font=("Arial", 14))
            entry_contact.pack()

            label_medical = Label(details_frame, text="Medical Issues:", font=("Arial", 16, "bold"))
            label_medical.pack()
            entry_medical = Entry(details_frame, font=("Arial", 14))
            entry_medical.pack()

            def send_report():
                name = entry_name.get().strip()
                contact = entry_contact.get().strip()
                medical_issues = entry_medical.get().strip()

                if not name or not contact or not medical_issues:
                    messagebox.showerror("Error", "Please enter all details.")
                    return

                messagebox.showinfo("Report Sent", "The report has been sent successfully!")
                details_window.destroy()

            button_send_report = ttk.Button(details_frame, text="Send Report to Therapist", style="CurvedButton.TButton", command=send_report)
            button_send_report.pack()

            details_window.mainloop()

        button_send_to_therapist = ttk.Button(report_frame, text="Send Report", style="CurvedButton.TButton", command=send_report_to_therapist)
        button_send_to_therapist.pack()

        report_window.mainloop()

    button_end = ttk.Button(button_frame, text="End Detection", style="CurvedButton.TButton", command=stop_detection)
    button_end.pack(side='left')

    button_report = ttk.Button(button_frame, text="Generate Report", style="CurvedButton.TButton", command=generate_report)
    button_report.pack(side='left')

    label_img = Label(window2)
    label_img.pack()

    while not stop_camera:
        ret, frame = cap.read()
        frame = cv2.resize(frame, (640, 480))
        if not ret:
            break
        frame = cv2.flip(frame, 1)

        face_detector_front = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        gray_frame_front = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        num_faces = face_detector_front.detectMultiScale(gray_frame_front, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            roi_gray_frame_front = gray_frame_front[y:y + h, x:x + w]
            cropped_img_front = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame_front, (48, 48)), -1), 0)

            emotion_prediction_front = emotion_model.predict(cropped_img_front)
            maxindex_front = int(np.argmax(emotion_prediction_front))
            cv2.putText(frame, emotion_dict[maxindex_front], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            emotion_report.append(maxindex_front)

        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_tk = ImageTk.PhotoImage(image=img)
        label_img.configure(image=img_tk)
        label_img.image = img_tk

        window2.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

window = Tk()
window.title("Emotion Detection")

background_image = Image.open("background_image.jpg")
background_image = background_image.resize((1500, 1000), Image.LANCZOS)
background_photo = ImageTk.PhotoImage(background_image)

label_background = Label(window, image=background_photo)
label_background.place(x=0, y=0, relwidth=1, relheight=1)
label_background.image = background_photo

button_frame = Frame(window, bg='white')
button_frame.pack(side='bottom', pady=20)

label_heading = Label(window, text="Emotion Recognition", font=("Century Gothic", 30, "bold"))
label_heading.place(relx=0.5, rely=0.5, anchor='center')

style = ttk.Style()
style.configure(
    "CurvedButton.TButton",
    font=("Arial", 14, "bold"),
    relief="flat",
    background="#0f3057",
    foreground="black"
)

button_start = ttk.Button(button_frame, text="Identify Mood", style="CurvedButton.TButton", command=start_detection)
button_start.pack(side='left')

window.mainloop()

1/1 [==============================] - 0s 98ms/step


In [4]:
import cv2
import numpy as np
from tkinter import Tk, Button, Toplevel, Label, messagebox, Frame, filedialog, Entry
from PIL import Image, ImageTk
from keras.models import model_from_json
from tkinter import ttk

# Load JSON and create model
json_file = open('model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)

# Load weights into the model
emotion_model.load_weights("model/emotion_model.h5")

emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}

emotion_report = []  # Variable to store emotion predictions

def end_detection():
    window2.destroy()
    window.deiconify()

def start_detection():
    global cap, window2, label_img, stop_camera, emotion_report

    window.withdraw()  # Hide the first window

    cap = cv2.VideoCapture(0)
    stop_camera = False
    emotion_report = []  # Reset emotion report

    window2 = Toplevel()
    window2.title("Emotion Detection")

    background_image = Image.open("background_image.jpg")
    background_image = background_image.resize((1400, 1000), Image.LANCZOS)
    background_photo = ImageTk.PhotoImage(background_image)

    label_background = Label(window2, image=background_photo)
    label_background.place(x=0, y=0, relwidth=1, relheight=1)
    label_background.image = background_photo

    button_frame = Frame(window2, bg='white', padx=20, pady=20)
    button_frame.pack(side='bottom')

    def stop_detection():
        global stop_camera
        stop_camera = True
        window2.destroy()
        window.deiconify()

    def generate_report():
        global stop_camera, emotion_report
        stop_camera = True

        if not emotion_report:
            messagebox.showerror("Error", "No emotions detected. Please try again.")
            return

        emotion_count = {}
        total_emotions = len(emotion_report)

        for emotion in emotion_report:
            if emotion in emotion_count:
                emotion_count[emotion] += 1
            else:
                emotion_count[emotion] = 1

        emotion_percentage = {}
        for emotion, count in emotion_count.items():
            percentage = (count / total_emotions) * 100
            emotion_percentage[emotion] = percentage

        overall_mood = max(emotion_percentage, key=emotion_percentage.get)

        report_window = Toplevel()
        report_window.title("Emotion Report")

        # Set the background image for the report window
        background_image = Image.open("background_image.jpg")
        background_image = background_image.resize((1500, 1000), Image.LANCZOS)
        background_photo = ImageTk.PhotoImage(background_image)
        label_background = Label(report_window, image=background_photo)
        label_background.place(x=0, y=0, relwidth=1, relheight=1)
        label_background.image = background_photo  # Attach the image object to the label to prevent it from being garbage collected

        report_frame = Frame(report_window)
        report_frame.pack()

        for emotion, percentage in emotion_percentage.items():
            label = Label(report_frame, text=f"{emotion_dict[emotion]}: {percentage:.2f}%", font=("Arial", 16, "bold"))
            label.pack(anchor='center')

        label_overall_mood = Label(report_frame, text=f"Overall Mood: {emotion_dict[overall_mood]}", font=("Arial", 20, "bold"))
        label_overall_mood.pack(anchor='center')

        def send_report_to_therapist():
            report_window.destroy()
            details_window = Toplevel()
            details_window.title("Patient Details")

            # Set the background image for the details window
            background_image = Image.open("background_image.jpg")
            background_image = background_image.resize((1500, 1000), Image.LANCZOS)
            background_photo = ImageTk.PhotoImage(background_image)
            label_background = Label(details_window, image=background_photo)
            label_background.place(x=0, y=0, relwidth=1, relheight=1)
            label_background.image = background_photo

            details_frame = Frame(details_window)
            details_frame.pack()

            # Add labels and entry fields for patient details
            label_name = Label(details_frame, text="Name:", font=("Arial", 16, "bold"))
            label_name.pack()
            entry_name = Entry(details_frame, font=("Arial", 14))
            entry_name.pack()

            label_contact = Label(details_frame, text="Contact No:", font=("Arial", 16, "bold"))
            label_contact.pack()
            entry_contact = Entry(details_frame, font=("Arial", 14))
            entry_contact.pack()

            label_medical = Label(details_frame, text="Medical Issues:", font=("Arial", 16, "bold"))
            label_medical.pack()
            entry_medical = Entry(details_frame, font=("Arial", 14))
            entry_medical.pack()

            def send_report():
                name = entry_name.get().strip()
                contact = entry_contact.get().strip()
                medical_issues = entry_medical.get().strip()

                if not name or not contact or not medical_issues:
                    messagebox.showerror("Error", "Please enter all details.")
                    return

                messagebox.showinfo("Report Sent", "The report has been sent successfully!")
                details_window.destroy()

            button_send_report = ttk.Button(details_frame, text="Send Report to Therapist", style="CurvedButton.TButton", command=send_report)
            button_send_report.pack()

            details_window.mainloop()

        button_send_to_therapist = ttk.Button(report_frame, text="Send Report", style="CurvedButton.TButton", command=send_report_to_therapist)
        button_send_to_therapist.pack()

        report_window.mainloop()

    button_end = ttk.Button(button_frame, text="End Detection", style="CurvedButton.TButton", command=stop_detection)
    button_end.pack(side='left')

    button_report = ttk.Button(button_frame, text="Generate Report", style="CurvedButton.TButton", command=generate_report)
    button_report.pack(side='left')

    label_img = Label(window2)
    label_img.pack()

    while not stop_camera:
        ret, frame = cap.read()
        frame = cv2.resize(frame, (640, 480))
        if not ret:
            break
        frame = cv2.flip(frame, 1)

        face_detector_front = cv2.CascadeClassifier('haarcascades/haarcascade_frontalface_default.xml')
        gray_frame_front = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        num_faces = face_detector_front.detectMultiScale(gray_frame_front, scaleFactor=1.3, minNeighbors=5)

        for (x, y, w, h) in num_faces:
            cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
            roi_gray_frame_front = gray_frame_front[y:y + h, x:x + w]
            cropped_img_front = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame_front, (48, 48)), -1), 0)

            emotion_prediction_front = emotion_model.predict(cropped_img_front)
            maxindex_front = int(np.argmax(emotion_prediction_front))
            cv2.putText(frame, emotion_dict[maxindex_front], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            emotion_report.append(maxindex_front)

        img = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        img_tk = ImageTk.PhotoImage(image=img)
        label_img.configure(image=img_tk)
        label_img.image = img_tk

        window2.update()

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

window = Tk()
window.title("Emotion Detection")

background_image = Image.open("background_image.jpg")
background_image = background_image.resize((1500, 1000), Image.LANCZOS)
background_photo = ImageTk.PhotoImage(background_image)

label_background = Label(window, image=background_photo)
label_background.place(x=0, y=0, relwidth=1, relheight=1)
label_background.image = background_photo

button_frame = Frame(window, bg='white')
button_frame.pack(side='bottom', pady=20)

label_heading = Label(window, text="Emotion Recognition", font=("Century Gothic", 30, "bold"))
label_heading.place(relx=0.5, rely=0.5, anchor='center')

style = ttk.Style()
style.configure(
    "CurvedButton.TButton",
    font=("Arial", 14, "bold"),
    relief="flat",
    background="#0f3057",
    foreground="black"
)

button_start = ttk.Button(button_frame, text="Identify Mood", style="CurvedButton.TButton", command=start_detection)
button_start.pack(side='left')

window.mainloop()

1/1 [==============================] - 0s 80ms/step


In [5]:
import matplotlib.pyplot as plt

# Prepare data for pie chart
emotions = list(emotion_percentage.keys())  # Extract emotions from the dictionary
percentages = list(emotion_percentage.values())  # Extract percentages from the dictionary

# Plot pie chart
plt.figure(figsize=(8, 6))  # Set the figure size (width=8, height=6)
plt.pie(percentages, labels=emotions, autopct='%1.1f%%')  # Plot the pie chart with percentages and emotion labels
plt.title('Emotion Report')  # Set the title of the pie chart

# Display the pie chart
plt.show()
#In this code, we use the previously calculated emotion_percentage dictionary to extract the emotions and percentages. Then, we create a pie chart using plt.pie(), specifying the percentages as the data, labels as the emotions, and using the %1.1f%% format to display percentages with one decimal place. The plt.title() function sets the title of the pie chart to 'Emotion Report'. Finally, plt.show() is used to display the pie chart.

NameError: name 'emotion_percentage' is not defined